# Part 1: Data import and preparation



In [1]:
# Import the modules necessary for the program


import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
from shapely.geometry import Point, LineString, Polygon

%matplotlib inline
plt.ion()

In [2]:
# Creates matplotlib handles which will be used to create a legend of the features added on the maps
def generate_handles(labels, colors, edge='k', alpha=1):
    lc = len(colors)  
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc], edgecolor=edge, alpha=alpha))
    return handles

In [3]:
# Creates a scale (with a division at 0,5 and 10m)  located in the upper right corner
def scale_bar(ax, location=(0.32, 0.95)):
    x0, x1, y0, y1 = ax.get_extent()
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    ax.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=9, transform=ax.projection)
    ax.plot([sbx, sbx - 5000], [sby, sby], color='k', linewidth=6, transform=ax.projection)
    ax.plot([sbx-5000, sbx - 10000], [sby, sby], color='w', linewidth=6, transform=ax.projection)

    ax.text(sbx, sby-1000, '10 km', transform=ax.projection, fontsize=10)
    ax.text(sbx-5000, sby-1000, '5 km', transform=ax.projection, fontsize=10)
    ax.text(sbx-10000, sby-1000, '0 km', transform=ax.projection, fontsize=10)


In [ ]:
#Import the different data files used for the analyses

flood= gpd.read_file(os.path.abspath('Project_datafiles/Flood_2m.shp'))
roads= gpd.read_file(os.path.abspath('Project_datafiles/Fermanagh_roads.shp'))
buildings= gpd.read_file(os.path.abspath('Project_datafiles/Building_Fermanagh.shp'))
pop_demography=gpd.read_file(os.path.abspath('Project_datafiles/popdemography.csv'))
outline = gpd.read_file(os.path.abspath('Project_datafiles/Fermanagh_DCA.shp'))
small_area= gpd.read_file(os.path.abspath('Project_datafiles/SApoly.shp'))
land_cover= gpd.read_file(os.path.abspath('Project_datafiles/LC_Fermanagh.shp'))

In [ ]:
#Merge the pop_demography csv with the small_area shapefile using the SA2011 column as the common column
small_area =gpd.GeoDataFrame(pop_demography.merge(small_area, on="SA2011"))
small_area.head()

In [ ]:
#Set the geometry column and edit the column name to geometry
small_area.rename(columns={'geometry_y':'geometry'}, inplace=True)

small_area.set_geometry('geometry')

In [ ]:
# Set the columns data type to integer instead of string
small_area['residents'] = small_area['residents'].astype(int)
small_area['Shape_Area'] = small_area['Shape_Area'].astype(int)
small_area['elderly'] = small_area['elderly'].astype(int)
small_area['children'] = small_area['children'].astype(int)

In [ ]:
# Create a population density, percentage of elderly and percentage of children columns and show the new geodatabase

for ind, row in small_area.iterrows(): 
    small_area.loc[ind, 'pop_density'] = row['residents']/ row['Shape_Area']* 1000000


for ind, row in small_area.iterrows(): 
    small_area.loc[ind, 'per_elderly'] = row['elderly']/ row['residents']* 100
    
for ind, row in small_area.iterrows():
    small_area.loc[ind, 'per_children'] = row['children']/ row['residents']* 100
    
print(small_area.head())

In [ ]:
# Set the Geographic Coordinate system to the Irish Transverse Mercator for all shapefiles
flood.to_crs(epsg = 2157)
roads.to_crs(epsg = 2157)
buildings.to_crs(epsg = 2157)
outline.to_crs(epsg = 2157)
small_area.to_crs(epsg = 2157)
land_cover.to_crs(epsg = 2157)

In [ ]:
# Create a function to subset the shapefiles based on the flood polygon
def flooded(shapefile):
    flood_geom = flood['geometry'].values[0]
    flooded = shapefile['geometry'].within(flood_geom)
    return shapefile['geometry'].within(flood_geom)


# Part 2: Impact on the landcover analyses and map

In [ ]:
#Create a new figure set to the Irish Transverse Mercator

myFig = plt.figure(figsize=(12, 12))

myCRS = ccrs.UTM(29)  
ax = plt.axes(projection=myCRS) 

In [ ]:
# Add the outline for Fermanagh County and zoom to the extent of the flood polygon
outline_feature = ShapelyFeature(outline['geometry'], myCRS, edgecolor='k', facecolor='w')
xmin, ymin, xmax, ymax = flood.total_bounds
ax.add_feature(outline_feature) # add the features we've created to the map.
ax.set_extent([xmin-1000, xmax+1000, ymin-1000, ymax+1000], crs=myCRS)


In [ ]:
# Check the number of Landcover type present in the dataset and print the names
landcover= len(land_cover.LAND_COVER.unique())
print('Number of unique features: {}'.format(landcover))
list_landcover = list(land_cover.LAND_COVER.unique())
print(list_landcover) 

In [ ]:
# Assign a color for each landcover type  and add the feature to the map
land_cover_colors = ['springgreen','olive', 'sienna','darkred', 'lawngreen', 'forestgreen','yellowgreen', 'y','darkgreen','darkorange','lightgrey','gold','black','grey']
landcover=list(land_cover.LAND_COVER.unique())
for ii, name in enumerate(landcover):
    feat = ShapelyFeature(land_cover.loc[land_cover['LAND_COVER'] == name, 'geometry'], # first argument is the geometry
                          myCRS, # second argument is the CRS
                          edgecolor='k', # outline the feature in black
                          facecolor=land_cover_colors[ii],
                        linewidth=0.1,
                         alpha=0.8) # set the face color to the corresponding color from the list
                           # set the outline width to be 1 pt
                           # set the alpha (transparency) to be 0.25 (out of 1)
    ax.add_feature(feat)


In [ ]:
# Add the flood polygon to the map
flood_feature = ShapelyFeature(flood['geometry'], myCRS, edgecolor='r',facecolor='navy',alpha=0.25,linewidth=0.25)
ax.add_feature(flood_feature)


In [ ]:
# Generate the legend label for the landcover and flood polygon
landcover_handles = generate_handles(land_cover.LAND_COVER.unique(), land_cover_colors)
flood_handles = generate_handles(['Flood'], ['navy'])

In [ ]:
# Add the legend, scale and gridlines to the map and prints the map 
handles = landcover_handles  + flood_handles
labels = landcover + ['Flood']

leg = ax.legend(handles, labels, title='Legend', title_fontsize=12,
                 fontsize=10, loc='lower left', frameon=True, framealpha=1)

gridlines = ax.gridlines(draw_labels=True, alpha=1, edgecolor='k')
                         
gridlines.left_labels = False 
gridlines.bottom_labels = False

scale_bar(ax)
myFig

#myFig.savefig('land_cover.png')

In [ ]:
# Subset the landcover data using the flood and shows the new dataset
land_cover_flooded = flooded(land_cover) 
land_flooded = land_cover[land_cover_flooded] 
print(land_flooded[['LAND_COVER','Shape_Area']])

In [ ]:
# Shows the total flooded area(sqm) of each landcover
land_flooded.groupby(['LAND_COVER'])['Shape_Area'].sum().sort_values(ascending=False)

# Part 3: Infrastructure and population analyses and map

In [ ]:
#Create a new figure set to the Irish Transverse Mercato

myCRS = ccrs.UTM(29)

myFig2, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection=myCRS)) 


In [ ]:
#Add the flood and Fermanagh county outline and set the extent to the flood polygon
flood_feature = ShapelyFeature(flood['geometry'], myCRS, edgecolor='k',facecolor='silver',alpha=0.5,linewidth=0.5)
ax.add_feature(flood_feature)
outline_feature = ShapelyFeature(outline['geometry'], myCRS, edgecolor='k', facecolor='w')
xmin, ymin, xmax, ymax = flood.total_bounds
ax.set_extent([xmin-1000, xmax+1000, ymin-1000, ymax+1000], crs=myCRS)


In [ ]:
# Add the road shapefile to the map
roads_feature = ShapelyFeature(roads['geometry'], myCRS, edgecolor='k', linewidth=1)
ax.add_feature(roads_feature)


In [ ]:
# Add the building shapefile and creates a different label for each CLASSIFICA attribute. Add the different labels to the legend
residential=buildings.loc[buildings['CLASSIFICA']=='Residential']
residential_handle= ax.plot(residential.geometry.x, residential.geometry.y, 's', color='b', ms=1, transform=myCRS)

commercial=buildings.loc[buildings['CLASSIFICA']=='Commercial']
commercial_handle=ax.plot(commercial.geometry.x, commercial.geometry.y, 'o', color='darkviolet', ms=2, transform=myCRS)

education=buildings.loc[buildings['CLASSIFICA']=='Education']
education_handle=ax.plot(education.geometry.x, education.geometry.y, '^', color='g', ms=6, transform=myCRS)

health=buildings.loc[buildings['CLASSIFICA']=='Health']
health_handle=ax.plot(health.geometry.x, health.geometry.y, '*', color='r', ms=6, transform=myCRS)

other=buildings.loc[buildings['CLASSIFICA']=='Other']
other_handle=ax.plot(other.geometry.x, other.geometry.y, 'h', color='limegreen', ms=1, transform=myCRS)



In [ ]:
#Add the population density, percentage of children and elderly and small_area shapefile to the map. Change which data is displayed  by removing and adding the #

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1, axes_class=plt.Axes)

small_area_plot=small_area.plot(column='pop_density', ax=ax, vmax=100, cmap='YlOrRd',
                       legend=True, cax=cax, legend_kwds={'label': 'Population Density'})

#small_area_plot=small_area.plot(column='per_elderly', ax=ax, vmax=30, cmap='YlOrRd',
                       #legend=True, cax=cax, legend_kwds={'label': 'Percentage of elderly'})

#small_area_plot=small_area.plot(column='per_children', ax=ax, vmax=30, cmap='YlOrRd',
                       #legend=True, cax=cax, legend_kwds={'label': 'Percentage of children'})

sa_outline = ShapelyFeature(small_area['geometry'], myCRS, edgecolor='r',linewidth=0.05, facecolor='none')
ax.add_feature(sa_outline)
myFig2

In [ ]:
# Generate the legend label for the road and flood features
roads_handle = [mlines.Line2D([], [], color='k')]
flood_handle = generate_handles(['Flood'], ['navy'])

In [ ]:
#Add the legend, scale and gridlines to the map and print the map
handles = roads_handle  + flood_handle + residential_handle + commercial_handle + other_handle + health_handle + education_handle # use '+' to concatenate (combine) lists
labels = ['Roads'] + ['Flood'] + ['Residential'] + ['Commercial'] + ['Other'] + ['Health'] + ['Education']

leg = ax.legend(handles, labels, title='Legend', title_fontsize=12,
                 fontsize=10, loc='lower left',markerscale=4, frameon=True, framealpha=1)


gridlines = ax.gridlines(draw_labels=True, alpha=1, edgecolor='k') 
                         
gridlines.right_labels = False 
gridlines.bottom_labels = False

scale_bar(ax)
myFig2

#myFig2.savefig('infrastructure_population.png')

In [ ]:
#Subset the different features using the flood polygon
build_flooded = flooded(buildings) 
building_flooded = buildings[build_flooded] 

rd_flooded = flooded(roads)
roads_flooded = roads[rd_flooded] 

sa_flooded = small_area.sjoin(flood, how="inner") 


In [ ]:
# Calculate the total length(in km) of flooded road and A Class road
sum_roads = roads_flooded['Length'].sum() /1000
sum_motorway = roads_flooded[roads_flooded['CLASS'] == 'A']['Length'].sum() /1000
print('{:.2f} total km of roads'.format(sum_roads))
print('{:.2f} total km of A class road'.format(sum_motorway))

In [ ]:
# Count the number of flooded buildings 
building_flooded.groupby(['CLASSIFICA'])['CLASSIFICA'].count().sort_values(ascending=False)

In [ ]:
# Show which flooded Small Area has the highest population density
high_density= sa_flooded[sa_flooded.pop_density==sa_flooded.pop_density.max()]
print(high_density[['SA2011', 'pop_density']])

In [ ]:
# Show which Small Area has a high percentage of elderly as well as a high percentage of children
sa_flooded.loc[(sa_flooded['per_elderly']>= sa_flooded['per_elderly'].mean())&(sa_flooded['per_children']>= sa_flooded['per_children'].mean())]



In [ ]:
# Show which Small Area has a low population density but a high percentage of children 
sa_flooded.loc[(sa_flooded['pop_density']<= sa_flooded['pop_density'].mean())&(sa_flooded['per_children']>= sa_flooded['per_children'].mean())]